# Overview

This notebook shows how to train and test a neural network on the BELLA data.

In order to run this notebook, you need to produce first produce CSV files for the training and testing data. (See the folder `experimental data`.)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from botorch.models.transforms.input import AffineInputTransform
import torch
from Neural_Net_Classes import NN as NN

<h2>Loading Split Data</h2>

In [2]:
# Load the training set
training_set_df = pd.read_csv('training_set_1.csv')

# Load the test set
test_set_df = pd.read_csv('test_set_1.csv')

<h2>Visualizing Split Datasets</h2>

In [ ]:
plt.clf()
ax = plt.figure().add_subplot(projection='3d')

ax.scatter( training_set_df['TOD (s^3)'], training_set_df['z_target (m)'],
           training_set_df['n_protons (1/sr)'], c='r', alpha=0.3, label='Training Set')
ax.scatter( test_set_df['TOD (s^3)'], test_set_df['z_target (m)'],
           test_set_df['n_protons (1/sr)'], c='b', alpha=0.3, label='Testing Set')
ax.view_init(elev=40., azim=40, roll=0)
plt.xlabel('TOD')
plt.ylabel('z_target')
plt.legend()

<h2>Normalizing Data</h2>

In [4]:
# Define the input and output normalizations, based on the training set
X = torch.tensor(training_set_df[['z_target (m)', 'TOD (s^3)']].values, dtype=torch.float)
input_transform = AffineInputTransform(2, coefficient=X.std(axis=0), offset=X.mean(axis=0))
y = torch.tensor(training_set_df['n_protons (1/sr)'].values, dtype=torch.float).reshape(-1,1)
output_transform = AffineInputTransform( 1, coefficient=y.std(axis=0), offset=y.mean(axis=0))


In [5]:
# Apply normalization to the training and test sets
norm_training_set_df = training_set_df.copy()
norm_training_set_df[['z_target (m)', 'TOD (s^3)']] = input_transform( torch.tensor( training_set_df[['z_target (m)', 'TOD (s^3)']].values ) )
norm_training_set_df[['n_protons (1/sr)']] = output_transform( torch.tensor( training_set_df[['n_protons (1/sr)']].values.reshape(-1,1) ) )

norm_test_set_df = test_set_df.copy()
norm_test_set_df[['z_target (m)', 'TOD (s^3)']] = input_transform( torch.tensor( test_set_df[['z_target (m)', 'TOD (s^3)']].values ) )
norm_test_set_df[['n_protons (1/sr)']] = output_transform( torch.tensor( test_set_df[['n_protons (1/sr)']].values.reshape(-1,1) ) )


<h2>Visualizing Normalized Data</h2>

In [ ]:
# Create a 3D plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter( norm_training_set_df['TOD (s^3)'], norm_training_set_df['z_target (m)'],
           norm_training_set_df['n_protons (1/sr)'], c='r', alpha=0.7, label='Training Set')
ax.scatter( norm_test_set_df['TOD (s^3)'], norm_test_set_df['z_target (m)'],
           norm_test_set_df['n_protons (1/sr)'], c='b', alpha=0.7, label='Testing Set')
ax.view_init(elev=40., azim=40, roll=0)
# Set labels and title
ax.set_xlabel('Normalized TOD')
ax.set_ylabel('Normalized Z')
ax.set_zlabel('Normalized Protons')

# Add legend
ax.legend()
# Show plot
plt.show()

<h1>Neural Network Framework</h1>

<h2>Build and Train Neural Networks</h2>

In [7]:
# Extract the normalized training and test sets
norm_z_train_set = torch.tensor(norm_training_set_df['z_target (m)'].values.reshape(-1,1), dtype=torch.float)
norm_TOD_train_set = torch.tensor(norm_training_set_df['TOD (s^3)'].values.reshape(-1,1), dtype=torch.float)
norm_protons_train_set = torch.tensor(norm_training_set_df['n_protons (1/sr)'].values.reshape(-1,1), dtype=torch.float)
norm_z_test_set = torch.tensor(norm_test_set_df['z_target (m)'].values.reshape(-1,1), dtype=torch.float)
norm_TOD_test_set = torch.tensor(norm_test_set_df['TOD (s^3)'].values.reshape(-1,1), dtype=torch.float)
norm_protons_test_set = torch.tensor(norm_test_set_df['n_protons (1/sr)'].values.reshape(-1,1), dtype=torch.float)

In [ ]:
net = NN()
net.train_model(norm_z_train_set, norm_TOD_train_set, norm_protons_train_set)
net.plot_loss()
net.test_model(norm_z_test_set, norm_TOD_test_set, norm_protons_test_set)

In [9]:
train_predictions = net.predict(norm_z_train_set, norm_TOD_train_set)
test_predictions = net.predict(norm_z_test_set, norm_TOD_test_set)

<h2>Plotting Predictions</h2>

In [ ]:
fig, ax = plt.subplots()

ax.scatter(norm_z_train_set, norm_protons_train_set, label='Training Set 1')
ax.scatter(norm_z_test_set, norm_protons_test_set, label='Test Set 1')

ax.scatter(train_predictions['Z_target'], train_predictions['predictions'], label='predictions', s=50, facecolors='none', edgecolors='r')
ax.scatter(test_predictions['Z_target'], test_predictions['predictions'], s=50, facecolors='none', edgecolors='r')

plt.title("n_protons predictions")
plt.xlabel('z_target (m)')
plt.ylabel('Number of protons (1/sr)')
plt.legend()

In [ ]:
# Create a 3D plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Scatter plot for training set
ax.scatter(norm_TOD_train_set, norm_z_train_set, norm_protons_train_set, label='Training Set 1', alpha=0.7)
ax.scatter(norm_TOD_test_set, norm_z_test_set, norm_protons_test_set, label='Test Set 1', alpha=0.7)

ax.scatter(train_predictions['TOD'], train_predictions['Z_target'], train_predictions['predictions'], label='predictions 1', s=50, facecolors='none', edgecolors='r')
ax.scatter(test_predictions['TOD'], test_predictions['Z_target'], test_predictions['predictions'], s=50, facecolors='none', edgecolors='r')

ax.view_init(elev=40., azim=40, roll=0)
# Set labels and title
ax.set_title('Experimental Data v Predictions')
ax.set_xlabel('TOD')
ax.set_ylabel('z_target')
ax.set_zlabel('n Protons')

# Add legend
ax.legend()
# Show plot
plt.show()

# Save the model as a LUME model

In [12]:
from lume_model.models import TorchModel
from lume_model.variables import ScalarInputVariable, ScalarOutputVariable
model = TorchModel

In [77]:
input_variables = [
    ScalarInputVariable(name="z_target", default=0, value_range=[-1,1]),
    ScalarInputVariable(name="TOD", default=0, value_range=[-1,1])
]
output_variables = [
    ScalarOutputVariable(name="n_protons", default=0, value_range=[0,10])
]

model = TorchModel(
    model=net,
    input_variables=input_variables,
    output_variables=output_variables,
    input_transformers=[input_transform],
    output_transformers=[output_transform]
)

In [95]:
model.dump( file='bella_saved_model.yml' )

# Show LUME model prediction on unnormalized data

In [96]:
loaded_model = TorchModel('bella_saved_model.yml')

In [ ]:
plt.clf()
ax = plt.figure().add_subplot(projection='3d')

ax.scatter( training_set_df['TOD (s^3)'], training_set_df['z_target (m)'],
           training_set_df['n_protons (1/sr)'], alpha=0.7, label='Training Set')
ax.scatter( test_set_df['TOD (s^3)'], test_set_df['z_target (m)'],
           test_set_df['n_protons (1/sr)'], alpha=0.7, label='Testing Set')

z_target = torch.tensor( training_set_df['z_target (m)'].values )
TOD = torch.tensor( training_set_df['TOD (s^3)'].values )
n_predict = loaded_model.evaluate( {'z_target': z_target, 'TOD': TOD} )
ax.scatter( training_set_df['TOD (s^3)'], training_set_df['z_target (m)'],
           n_predict['n_protons'], s=50, facecolors='none', edgecolors='r', label='Training Set')

z_target = torch.tensor( test_set_df['z_target (m)'].values )
TOD = torch.tensor( test_set_df['TOD (s^3)'].values )
n_predict = loaded_model.evaluate( {'z_target': z_target, 'TOD': TOD} )
ax.scatter( test_set_df['TOD (s^3)'], test_set_df['z_target (m)'],
           n_predict['n_protons'], s=50, facecolors='none', edgecolors='r', label='Training Set')


ax.view_init(elev=40., azim=40, roll=0)
plt.xlabel('TOD')
plt.ylabel('z_target')
plt.legend()